In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import time

from sklearn.linear_model.stochastic_gradient import SGDRegressor

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from math import sqrt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
#Importing dataset
df = pd.read_csv("dataset/train_rand.csv", index_col=0)
df_val = pd.read_csv("dataset/val1_rand.csv", index_col=0)

In [3]:
print(df.shape)
print(df_val.shape)

(356284, 46)
(40291, 46)


In [4]:
df = df.drop(['Date', 'StoreID'], axis=1)
df_val = df_val.drop(['Date', 'StoreID'], axis=1)

We try to find the best number of features using PCA.

In [5]:
y = df['NumberOfSales']
y_val = df_val['NumberOfSales']
df = df.drop(['NumberOfSales'], axis=1)
df_val = df_val.drop(['NumberOfSales'], axis=1)

In [6]:
X_normalized = StandardScaler().fit_transform(df)
X_val_normalized = StandardScaler().fit_transform(df_val)

In [7]:
var_ratio = 0
n_c = 24

while var_ratio < 0.95:
    n_c += 1
    pca = PCA(n_components=n_c)
    new_data = pca.fit_transform(X_normalized)
    var_ratio = sum(pca.explained_variance_ratio_)
    print("Number of components: " + str(n_c) + ", total explained variance: " + str(var_ratio))

Number of components: 25, total explained variance: 0.9293112457922418
Number of components: 26, total explained variance: 0.942158956686758
Number of components: 27, total explained variance: 0.9542224697328177


In [9]:
cs = []
for i in range(1, n_c + 1):
    cs.append("Principal component " + str(i))
X = pd.DataFrame(data = new_data, columns = cs)

new_data_val = pca.fit_transform(X_val_normalized)
X_val = pd.DataFrame(data = new_data_val, columns = cs)

In [10]:
dict_res = {'loss_function' : [], 'alpha' : [], 'epsilon' : [], 'learning_rate' : [], 'mse' : [], 'mae' : []}

In [13]:
loss_fun = ['squared_loss', 'epsilon_insensitive']
learning_r = ['constant', 'optimal', 'invscaling']

start = time.time()

for l_f in loss_fun:
    for l_r in learning_r:
        for a in np.linspace(.0001, .01 , 20):
            for e in np.linspace(.1, .5, 5):
                sgd = SGDRegressor(loss=l_f, alpha=a, epsilon=e, learning_rate=l_r, shuffle=False, random_state=1234567890)
                dict_res['loss_function'].append(l_f)
                dict_res['alpha'].append(a)
                dict_res['epsilon'].append(e)
                dict_res['learning_rate'].append(l_r)
                sgd.fit(X, y)
                y_pred = sgd.predict(X_val)
                mse = sqrt(mean_squared_error(y_val,y_pred))
                mae = mean_absolute_error(y_val, y_pred)
                dict_res['mse'].append(mse)
                dict_res['mae'].append(mae)
                
                print(l_f, l_r, a, e, mse, mae)
            
end = time.time() - start
end

squared_loss constant 0.0001 0.1 219761981892.09692 95983344942.60895
squared_loss constant 0.0001 0.2 219761981892.09692 95983344942.60895
squared_loss constant 0.0001 0.30000000000000004 219761981892.09692 95983344942.60895
squared_loss constant 0.0001 0.4 219761981892.09692 95983344942.60895
squared_loss constant 0.0001 0.5 219761981892.09692 95983344942.60895
squared_loss constant 0.0006210526315789474 0.1 234572844144.35995 139389645103.443
squared_loss constant 0.0006210526315789474 0.2 234572844144.35995 139389645103.443
squared_loss constant 0.0006210526315789474 0.30000000000000004 234572844144.35995 139389645103.443
squared_loss constant 0.0006210526315789474 0.4 234572844144.35995 139389645103.443
squared_loss constant 0.0006210526315789474 0.5 234572844144.35995 139389645103.443
squared_loss constant 0.0011421052631578948 0.1 242503784158.45404 107580152395.59572
squared_loss constant 0.0011421052631578948 0.2 242503784158.45404 107580152395.59572
squared_loss constant 0.00

squared_loss constant 0.01 0.1 216206852795.11774 97717437357.94078
squared_loss constant 0.01 0.2 216206852795.11774 97717437357.94078
squared_loss constant 0.01 0.30000000000000004 216206852795.11774 97717437357.94078
squared_loss constant 0.01 0.4 216206852795.11774 97717437357.94078
squared_loss constant 0.01 0.5 216206852795.11774 97717437357.94078
squared_loss optimal 0.0001 0.1 57428245688.032135 24140776978.189396
squared_loss optimal 0.0001 0.2 57428245688.032135 24140776978.189396
squared_loss optimal 0.0001 0.30000000000000004 57428245688.032135 24140776978.189396
squared_loss optimal 0.0001 0.4 57428245688.032135 24140776978.189396
squared_loss optimal 0.0001 0.5 57428245688.032135 24140776978.189396
squared_loss optimal 0.0006210526315789474 0.1 1674.0591373269274 1316.3232635050356
squared_loss optimal 0.0006210526315789474 0.2 1674.0591373269274 1316.3232635050356
squared_loss optimal 0.0006210526315789474 0.30000000000000004 1674.0591373269274 1316.3232635050356
squared

squared_loss optimal 0.009478947368421052 0.2 1566.6294837361168 1233.47294930877
squared_loss optimal 0.009478947368421052 0.30000000000000004 1566.6294837361168 1233.47294930877
squared_loss optimal 0.009478947368421052 0.4 1566.6294837361168 1233.47294930877
squared_loss optimal 0.009478947368421052 0.5 1566.6294837361168 1233.47294930877
squared_loss optimal 0.01 0.1 1566.7180269509354 1233.263566317855
squared_loss optimal 0.01 0.2 1566.7180269509354 1233.263566317855
squared_loss optimal 0.01 0.30000000000000004 1566.7180269509354 1233.263566317855
squared_loss optimal 0.01 0.4 1566.7180269509354 1233.263566317855
squared_loss optimal 0.01 0.5 1566.7180269509354 1233.263566317855
squared_loss invscaling 0.0001 0.1 1591.3359378279533 1258.453480947638
squared_loss invscaling 0.0001 0.2 1591.3359378279533 1258.453480947638
squared_loss invscaling 0.0001 0.30000000000000004 1591.3359378279533 1258.453480947638
squared_loss invscaling 0.0001 0.4 1591.3359378279533 1258.453480947638
s

squared_loss invscaling 0.008957894736842106 0.1 1570.8785709062645 1240.691084251761
squared_loss invscaling 0.008957894736842106 0.2 1570.8785709062645 1240.691084251761
squared_loss invscaling 0.008957894736842106 0.30000000000000004 1570.8785709062645 1240.691084251761
squared_loss invscaling 0.008957894736842106 0.4 1570.8785709062645 1240.691084251761
squared_loss invscaling 0.008957894736842106 0.5 1570.8785709062645 1240.691084251761
squared_loss invscaling 0.009478947368421052 0.1 1569.9030124136157 1239.7880527561601
squared_loss invscaling 0.009478947368421052 0.2 1569.9030124136157 1239.7880527561601
squared_loss invscaling 0.009478947368421052 0.30000000000000004 1569.9030124136157 1239.7880527561601
squared_loss invscaling 0.009478947368421052 0.4 1569.9030124136157 1239.7880527561601
squared_loss invscaling 0.009478947368421052 0.5 1569.9030124136157 1239.7880527561601
squared_loss invscaling 0.01 0.1 1568.9489245329778 1238.899646841408
squared_loss invscaling 0.01 0.2 

epsilon_insensitive constant 0.007394736842105264 0.5 2179.9210642290845 1545.8810130781096
epsilon_insensitive constant 0.00791578947368421 0.1 2186.628374223192 1550.8656170050826
epsilon_insensitive constant 0.00791578947368421 0.2 2186.6258320617126 1550.8651699596753
epsilon_insensitive constant 0.00791578947368421 0.30000000000000004 2186.62384693134 1550.8651572843903
epsilon_insensitive constant 0.00791578947368421 0.4 2186.6238446479388 1550.8651546787125
epsilon_insensitive constant 0.00791578947368421 0.5 2186.6197079468207 1550.8649930488734
epsilon_insensitive constant 0.008436842105263158 0.1 2192.7594917805754 1555.4161195885906
epsilon_insensitive constant 0.008436842105263158 0.2 2192.747408544706 1555.413926150356
epsilon_insensitive constant 0.008436842105263158 0.30000000000000004 2192.7467804542207 1555.4137638944671
epsilon_insensitive constant 0.008436842105263158 0.4 2192.7479976233244 1555.4145643549002
epsilon_insensitive constant 0.008436842105263158 0.5 2192

epsilon_insensitive optimal 0.006352631578947369 0.5 3945.2701635509998 3260.8769672500757
epsilon_insensitive optimal 0.0068736842105263166 0.1 4041.6119985718055 3372.959770929839
epsilon_insensitive optimal 0.0068736842105263166 0.2 4041.6124820904033 3372.960016022142
epsilon_insensitive optimal 0.0068736842105263166 0.30000000000000004 4041.612483905932 3372.960364765067
epsilon_insensitive optimal 0.0068736842105263166 0.4 4041.6100405161405 3372.95747279496
epsilon_insensitive optimal 0.0068736842105263166 0.5 4041.609748221819 3372.957168501068
epsilon_insensitive optimal 0.007394736842105264 0.1 4125.972141748788 3471.0519409424282
epsilon_insensitive optimal 0.007394736842105264 0.2 4125.972060792798 3471.052104094696
epsilon_insensitive optimal 0.007394736842105264 0.30000000000000004 4125.972248997604 3471.0523763983033
epsilon_insensitive optimal 0.007394736842105264 0.4 4125.972228871636 3471.0525934653924
epsilon_insensitive optimal 0.007394736842105264 0.5 4125.97230526

epsilon_insensitive invscaling 0.005310526315789474 0.4 4814.315541151803 4262.1967763538105
epsilon_insensitive invscaling 0.005310526315789474 0.5 4814.315541151803 4262.1967763538105
epsilon_insensitive invscaling 0.005831578947368421 0.1 4814.290305517257 4262.196769577262
epsilon_insensitive invscaling 0.005831578947368421 0.2 4814.290305517257 4262.196769577262
epsilon_insensitive invscaling 0.005831578947368421 0.30000000000000004 4814.290305517257 4262.196769577262
epsilon_insensitive invscaling 0.005831578947368421 0.4 4814.290305517257 4262.196769577262
epsilon_insensitive invscaling 0.005831578947368421 0.5 4814.290305517257 4262.196769577262
epsilon_insensitive invscaling 0.006352631578947369 0.1 4814.271390943739 4262.19676466933
epsilon_insensitive invscaling 0.006352631578947369 0.2 4814.271390943739 4262.19676466933
epsilon_insensitive invscaling 0.006352631578947369 0.30000000000000004 4814.271390943739 4262.19676466933
epsilon_insensitive invscaling 0.0063526315789473

144.472234249115

In [21]:
for k in dict_res.keys():
        print(len(dict_res[k]))

813
813
813
813
812
812


In [18]:
dict_res

{'loss_function': ['squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_loss',
  'squared_